In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/nyc-rolling-sales.csv')

In [4]:
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [5]:
df=df.drop(columns=['Unnamed: 0'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12792 entries, 0 to 12791
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         12792 non-null  int64 
 1   NEIGHBORHOOD                    12792 non-null  object
 2   BUILDING CLASS CATEGORY         12792 non-null  object
 3   TAX CLASS AT PRESENT            12792 non-null  object
 4   BLOCK                           12792 non-null  int64 
 5   LOT                             12792 non-null  int64 
 6   EASE-MENT                       12792 non-null  object
 7   BUILDING CLASS AT PRESENT       12792 non-null  object
 8   ADDRESS                         12792 non-null  object
 9   APARTMENT NUMBER                12792 non-null  object
 10  ZIP CODE                        12792 non-null  int64 
 11  RESIDENTIAL UNITS               12792 non-null  int64 
 12  COMMERCIAL UNITS                12792 non-null

In [7]:
df.drop('EASE-MENT', axis=1, inplace=True)

In [8]:
contagem = df.applymap(lambda x: str(x).count("-"))
contagem_space = df.applymap(lambda x: str(x).count(" "))
print(f"there is {contagem.sum().sum()} '-' and {contagem_space.sum().sum()} spaces")

there is 66532 '-' and 377970 spaces


In [9]:
map_dict={'-':np.nan,' ':np.nan}
df.replace(map_dict,inplace = True)

In [10]:
sum(df.duplicated())


162

In [11]:
df.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING CLASS CATEGORY              0
TAX CLASS AT PRESENT               237
BLOCK                                0
LOT                                  0
BUILDING CLASS AT PRESENT          237
ADDRESS                              0
APARTMENT NUMBER                  6645
ZIP CODE                             0
RESIDENTIAL UNITS                    0
COMMERCIAL UNITS                     0
TOTAL UNITS                          0
LAND SQUARE FEET                     0
GROSS SQUARE FEET                    0
YEAR BUILT                           0
TAX CLASS AT TIME OF SALE            0
BUILDING CLASS AT TIME OF SALE       0
SALE PRICE                           1
SALE DATE                            1
dtype: int64

In [12]:
coluna_p_numerica = ['LAND SQUARE FEET', 'GROSS SQUARE FEET', 'SALE PRICE']
for col in df.columns:
    if col in coluna_p_numerica:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12792 entries, 0 to 12791
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         12792 non-null  int64  
 1   NEIGHBORHOOD                    12792 non-null  object 
 2   BUILDING CLASS CATEGORY         12792 non-null  object 
 3   TAX CLASS AT PRESENT            12555 non-null  object 
 4   BLOCK                           12792 non-null  int64  
 5   LOT                             12792 non-null  int64  
 6   BUILDING CLASS AT PRESENT       12555 non-null  object 
 7   ADDRESS                         12792 non-null  object 
 8   APARTMENT NUMBER                6147 non-null   object 
 9   ZIP CODE                        12792 non-null  int64  
 10  RESIDENTIAL UNITS               12792 non-null  int64  
 11  COMMERCIAL UNITS                12792 non-null  int64  
 12  TOTAL UNITS                     

In [14]:
# importação de valores que estão faltando em 'PÉS QUADRADOS DO TERRENO' e 'PÉS QUADRADOS BRUTOS' usando a mediana
for col in ['LAND SQUARE FEET', 'GROSS SQUARE FEET']:
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)

# Removendo linhas onde  'SALE PRICE' está faltando
df = df.dropna(subset=['SALE PRICE'])

# Verificando as alterações
df_info = {
    "Missing Values After Imputation": df[coluna_p_numerica].isnull().sum(),
    "Remaining Rows": len(df)
}

df_info

{'Missing Values After Imputation': LAND SQUARE FEET     0
 GROSS SQUARE FEET    0
 SALE PRICE           0
 dtype: int64,
 'Remaining Rows': 10113}

In [15]:
df.head(10)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,NaN,10009,5,0,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,NaN,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016-09-23 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,NaN,10009,6,0,6,2369.0,4615.0,1900,2,C2,8000000.0,2016-11-17 00:00:00
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,210 AVENUE B,NaN,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016-09-23 00:00:00
9,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,629 EAST 5TH STREET,NaN,10009,24,0,24,4489.0,18523.0,1920,2,D9,16232000.0,2016-11-07 00:00:00
11,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2B,400,21,D1,530-532 EAST 5TH STREET,NaN,10009,10,0,10,3717.0,12350.0,2009,2,D1,10350000.0,2016-10-17 00:00:00
12,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3 STREET, 1C",NaN,10009,0,0,0,2500.0,8574.0,1920,2,C6,1.0,2016-09-06 00:00:00
13,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3 STREET, 1C",NaN,10009,0,0,0,2500.0,8574.0,1920,2,C6,499000.0,2017-03-10 00:00:00
14,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3 STREET, 3A",NaN,10009,0,0,0,2500.0,8574.0,1920,2,C6,10.0,2017-04-28 00:00:00
15,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3RD STREET, 5A",NaN,10009,0,0,0,2500.0,8574.0,1920,2,C6,529500.0,2017-06-09 00:00:00


In [16]:
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

In [17]:
df['year'] = df['SALE DATE'].dt.year
df['month'] = df['SALE DATE'].dt.month

In [18]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,...,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,year,month
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,NaN,10009,...,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19,2017,7
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,NaN,10009,...,10,2272.0,6794.0,1913,2,C4,3936272.0,2016-09-23,2016,9
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,NaN,10009,...,6,2369.0,4615.0,1900,2,C2,8000000.0,2016-11-17,2016,11
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,210 AVENUE B,NaN,10009,...,8,1750.0,4226.0,1920,2,C4,3192840.0,2016-09-23,2016,9
9,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,629 EAST 5TH STREET,NaN,10009,...,24,4489.0,18523.0,1920,2,D9,16232000.0,2016-11-07,2016,11


In [19]:
df = df.drop(columns=['SALE DATE'])

In [26]:
df.head(11)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,year,month
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,C2,153 AVENUE B,10009,5,0,5,1633.0,6440.0,1900,2,C2,6625000.0,2017,7
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,C4,154 EAST 7TH STREET,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016,9
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,C2,301 EAST 10TH STREET,10009,6,0,6,2369.0,4615.0,1900,2,C2,8000000.0,2016,11
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,C4,210 AVENUE B,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016,9
9,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,D9,629 EAST 5TH STREET,10009,24,0,24,4489.0,18523.0,1920,2,D9,16232000.0,2016,11
11,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2B,400,21,D1,530-532 EAST 5TH STREET,10009,10,0,10,3717.0,12350.0,2009,2,D1,10350000.0,2016,10
12,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3 STREET, 1C",10009,0,0,0,2500.0,8574.0,1920,2,C6,1.0,2016,9
13,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3 STREET, 1C",10009,0,0,0,2500.0,8574.0,1920,2,C6,499000.0,2017,3
14,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3 STREET, 3A",10009,0,0,0,2500.0,8574.0,1920,2,C6,10.0,2017,4
15,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,373,40,C6,"327 EAST 3RD STREET, 5A",10009,0,0,0,2500.0,8574.0,1920,2,C6,529500.0,2017,6


In [25]:
# Normalmente O numero do apartamento é muito importante, mas como não terei acesso a essa informação,descartei
df = df.drop(columns='APARTMENT NUMBER', axis=1)